In [6]:
import pandas as pd
import requests
import json
import geopy

from geopy.geocoders import Nominatim

In [4]:
file_path = '/Users/Dracarys/Documents/data/'

In [5]:
trump_fillin = pd.read_csv(file_path + 'trump_location_filled.csv')

In [78]:
hillary_fillin = pd.read_csv(file_path + 'hillary_location_filled.csv')

In [3]:
trump_location_df = pd.read_csv(file_path + 'trump_location.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
trump_location_df.sample(5)

,user,date,id,text,userid,loc,state,lat,lon,hashtag
148902,TrumpChess,2016-10-30 10:23:06,792733572239024128,# GoHillary Hilarious Political Ads against,442426470,STL MO,MO,38.749603,-90.370219,"['#Abedin', '#Trump', '#Clinton', '#MeetThePre..."
423002,Biracial_Chick,2016-11-01 12:42:33,793493443427393536,"Sadly, DC is so corrupt. Nothing will come of ...",3410475429,Trump World 🌎,NaN,40.752520,-73.967689,['#DrainTheSwamp']
368503,yadkinfarm,2016-11-04 10:46:45,794551465046065152,The Hillary Clinton Takeover of the United Sta...,27903560,North Carolina,NaN,35.672964,-79.039292,"['#MAGA', '#TrumpPence16']"
777639,freedommary,2016-10-09 15:40:02,785203185199742976,stupid supporters insist will unite nation. Ho...,116855072,"Buffalo, NY.",NaN,42.886717,-78.878392,"['#Clinton', '#Hillary', '#USA', '#NeverHillar..."
708816,deemat727,2016-10-14 22:36:48,787120007805341696,# CrookedHillary,67614171,"Florida, USA",NaN,27.756767,-81.463983,"['#crookedmedia', '#crookedCNN', '#clintoncash..."


In [56]:
def set_state(dataframe):
    '''given a dataframe from groupby operation, change its state only if there is at least one state that is not nan'''
    if len(dataframe.state.dropna() ): # at least one row has state labelled
            if len(dataframe.state.isna()): # at least one row has state not labelled
                dataframe.loc[:, 'state'] = dataframe.state.dropna().unique()[0]
#                 dataframe.loc[:, 'loc'] = dataframe.loc.dropna().unique()[0]
    return dataframe

In [57]:
trump_fillin = trump_location_df.groupby(['lat', 'lon']).apply(set_state)

In [46]:
trump_fillin.state.isna().sum(), trump_location_df.state.isna().sum()

(135774, 546075)

In [73]:
trump_fillin.to_csv(file_path + 'trump_location_filled.csv', index=False)

In [47]:
hillary_location = pd.read_csv(file_path + 'hillary_location.csv', index=False)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
hillary_fillin = hillary_location.groupby(['lat', 'lon']).apply(set_state)

In [50]:
hillary_fillin.state.isna().sum(), hillary_location.state.isna().sum()

(59560, 218135)

In [82]:
hillary_fillin.to_csv(file_path + 'hillary_location_filled.csv', index=False)

In [51]:
state_dict = {}

In [52]:
geolocator = Nominatim()

In [53]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [75]:
def store_state(lat, lon):
    '''store state abbreviation if there are given lat and lon'''
    pair_str = '%f, %f' % (lat, lon)
    location = geolocator.reverse(pair_str)
    state = location.raw['address']['state']
    if state in us_state_abbrev:
        state = us_state_abbrev[state]
    if lat in state_dict:
        state_dict[lat][lon] = state
    else:
        state_dict[lat] = {lon: state}

In [76]:
for pair, dataframe in trump_fillin[trump_fillin.state.isna()].groupby(['lat', 'lon']):
    store_state(*pair)

GeocoderServiceError: HTTP Error 429: Too Many Requests

In [70]:
for lat in state_dict:
    for lon in state_dict[lat]:
        trump_fillin.loc[(trump_fillin.lat == lat).values & (trump_fillin.lon == lon).values, 'state'] = state_dict[lat][lon]

In [74]:
trump_fillin.state.isna().sum()

86817

In [79]:
for lat in state_dict:
    for lon in state_dict[lat]:
        hillary_fillin.loc[(hillary_fillin.lat == lat).values & (hillary_fillin.lon == lon).values, 'state'] = state_dict[lat][lon]

In [80]:
hillary_fillin.state.isna().sum()

50340

In [87]:
trump.state.isna().sum()

86817

In [85]:
hillary = pd.read_csv(file_path + 'hillary_location_filled.csv')

In [86]:
hillary.state.isna().sum()

50340